<a href="https://colab.research.google.com/github/shikhar00778/toxic-comment-classification/blob/master/Jigsaw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/12500/363832/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1559217188&Signature=WxgmexT%2FEzJj7SskfbmJohBGA9bbqMjz1wXJs%2BaoQvToWKmJAtbrCb4E1dE90Fm00PzRrXMN9tdIhaaRyyQrh650xv5Sjeiq%2BK3hR%2BJ2d3LHkp7Y2H9HVp8e2B9SQauyVgBXltiIc41Fkwp%2FkRccnpfhydbhnmymGhfoH6cCnomO9%2BNuRdBX7G6QjKoymmEzyXPYJyMXNMbVBvmaw%2BB4X%2F4MmZe%2FSecjCxVSz03drupVml6fE0ZUzSLbvxSEWLwP0hMzfWzyH9iMrEQkifMGyyBjHMGwnActYcjbqdOKH4U7HSUw1XDC%2FGrfp4SDhgVKGBBsiqFy7gRo8GXAH0HQXA%3D%3D&response-content-disposition=attachment%3B+filename%3Djigsaw-unintended-bias-in-toxicity-classification.zip" -O "jigsaw-unintended-bias-in-toxicity-classification.zip" -c

--2019-05-27 18:28:32--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/12500/363832/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1559217188&Signature=WxgmexT%2FEzJj7SskfbmJohBGA9bbqMjz1wXJs%2BaoQvToWKmJAtbrCb4E1dE90Fm00PzRrXMN9tdIhaaRyyQrh650xv5Sjeiq%2BK3hR%2BJ2d3LHkp7Y2H9HVp8e2B9SQauyVgBXltiIc41Fkwp%2FkRccnpfhydbhnmymGhfoH6cCnomO9%2BNuRdBX7G6QjKoymmEzyXPYJyMXNMbVBvmaw%2BB4X%2F4MmZe%2FSecjCxVSz03drupVml6fE0ZUzSLbvxSEWLwP0hMzfWzyH9iMrEQkifMGyyBjHMGwnActYcjbqdOKH4U7HSUw1XDC%2FGrfp4SDhgVKGBBsiqFy7gRo8GXAH0HQXA%3D%3D&response-content-disposition=attachment%3B+filename%3Djigsaw-unintended-bias-in-toxicity-classification.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c14::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299148730 (285M) [application/zip]
Saving to: ‘jigsaw-unintended-bias-

In [2]:
!unzip jigsaw-unintended-bias-in-toxicity-classification.zip

Archive:  jigsaw-unintended-bias-in-toxicity-classification.zip
  inflating: train.csv               
  inflating: sample_submission.csv   
  inflating: test.csv                


In [0]:
import pandas as pd
df = pd.read_csv('train.csv')

In [4]:
len(df)

1804874

In [5]:
!pip install git+https://github.com/kaushaltrivedi/fast-bert.git

  Cloning https://github.com/kaushaltrivedi/fast-bert.git to /tmp/pip-req-build-fw4xrb89
  Running command git clone -q https://github.com/kaushaltrivedi/fast-bert.git /tmp/pip-req-build-fw4xrb89
     |████████████████████████████████| 133kB 3.5MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-to0yy7q1/wheels/53/11/ef/67d507c8f4f05b843a6dd2c4f6360acdee4c79679c4ca36bc6
Successfully built fast-bert


In [6]:
%%writefile /usr/local/lib/python3.6/dist-packages/fast_bert/metrics.py
import numpy as np
from torch import Tensor
from sklearn.metrics import roc_curve, auc, hamming_loss, accuracy_score
import pdb

# def accuracy(out, labels):
#     outputs = np.argmax(out, axis=1)
#     return np.sum(outputs == labels)

def accuracy(y_pred:Tensor, y_true:Tensor):

    outputs = np.argmax(y_pred, axis=1)
    return np.mean(outputs.numpy() == y_true.detach().cpu().numpy())

def accuracy_multilabel(y_pred:Tensor, y_true:Tensor, sigmoid:bool=True):
    if sigmoid: y_pred = y_pred.sigmoid()
    outputs = np.argmax(y_pred, axis=1)
    real_vals = np.argmax(y_true, axis=1)
    return np.mean(outputs.numpy() == real_vals.numpy())

def accuracy_thresh(y_pred:Tensor, y_true:Tensor, thresh:float=0.5, sigmoid:bool=True):
    "Compute accuracy when `y_pred` and `y_true` are the same size."
    if sigmoid: y_pred = y_pred.sigmoid()
    return ((y_pred>thresh)==y_true.byte()).float().mean().item()
#     return np.mean(((y_pred>thresh)==y_true.byte()).float().cpu().numpy(), axis=1).sum()


def fbeta(y_pred:Tensor, y_true:Tensor, thresh:float=0.3, beta:float=2, eps:float=1e-9, sigmoid:bool=True):
    "Computes the f_beta between `preds` and `targets`"
    beta2 = beta ** 2
    if sigmoid: y_pred = y_pred.sigmoid()
    y_pred = (y_pred>thresh).float()
    y_true = y_true.float()
    TP = (y_pred*y_true).sum(dim=1)
    prec = TP/(y_pred.sum(dim=1)+eps)
    rec = TP/(y_true.sum(dim=1)+eps)
    res = (prec*rec)/(prec*beta2+rec+eps)*(1+beta2)
    return res.mean().item()

def roc_auc(y_pred: Tensor, y_true: Tensor):
    # ROC-AUC calcualation
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()
        
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    return roc_auc["micro"]


Overwriting /usr/local/lib/python3.6/dist-packages/fast_bert/metrics.py


In [7]:
!pip uninstall apex

In [0]:
!rm -rf apex

In [9]:
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!python setup.py install --cuda_ext --cpp_ext

Cloning into 'apex'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 4467 (delta 38), reused 40 (delta 20), pack-reused 4394
Receiving objects: 100% (4467/4467), 8.67 MiB | 20.60 MiB/s, done.
Resolving deltas: 100% (2866/2866), done.
/content/apex
torch.__version__  =  1.1.0

Compiling cuda extensions with
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
from /usr/local/cuda/bin

running install
running bdist_egg
running egg_info
creating apex.egg-info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing top-level names to apex.egg-info/top_level.txt
writing manifest file 'apex.egg-info/SOURCES.txt'
writing manifest file 'apex.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_li

In [10]:
cd ..

/content


In [11]:
import torch
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing import text,sequence
from tqdm import tqdm
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
from fastai.train import Learner
from fastai.train import DataBunch
from fastai.callbacks import *
from fastai.metrics import *
import os
import random

import apex
import pytorch_pretrained_bert
from pytorch_pretrained_bert import BertModel
from pytorch_pretrained_bert.tokenization import BertTokenizer

from fast_bert.data import BertDataBunch
from fast_bert.learner import BertLearner
from fast_bert.metrics import accuracy , accuracy_multilabel , roc_auc

Using TensorFlow backend.


In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 2501532.66B/s]


In [0]:
device = torch.device('cuda')

# check if multiple GPUs are available
if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

In [14]:
len(df)

1804874

In [0]:
df_valid = df[1700000:]

In [0]:
df_train = df[:1700000]

In [0]:
df_train = df_train.rename(index=str, columns={"comment_text": "text"})

In [0]:
df_valid = df_valid.rename(index=str, columns={"comment_text": "text"})

In [0]:

df_train.loc[df_train['target'] >= 0.5 , ['target']] = 1
df_train.loc[df_train['target'] < 0.5 , ['target']] = 0


df_valid.loc[df_valid['target'] >= 0.5 , ['target']] = 1
df_valid.loc[df_valid['target'] < 0.5 , ['target']] = 0

In [0]:
df_train.to_csv('train_sample.csv' , index = False)

In [0]:
df_valid.to_csv('valid.csv' , index = False)

In [0]:
l = ['target','severe_toxicity', 'obscene','identity_attack', 'insult', 'threat']
df_label = pd.DataFrame(l)

df_label.to_csv('labels.csv' , index = False , header = None)

In [0]:

args = {
    "max_seq_length": 128,
    "do_lower_case": True,
    "train_batch_size": 32,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

In [0]:
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [25]:
df.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count'],
      dtype='object')

In [0]:
databunch = BertDataBunch('/content', '/content', tokenizer, 
                          train_file='train_sample.csv', val_file='valid.csv',label_file='labels.csv',
                          label_col=['target','severe_toxicity', 'obscene','identity_attack', 'insult', 'threat'],
                          bs=args['train_batch_size'], maxlen=args['max_seq_length'], 
                          multi_gpu=multi_gpu, multi_label=True)

In [0]:
databunch.save(filename = "seq-128_batch-32.pkl")

In [0]:
metrics = []
metrics.append({'name': 'roc_auc', 'function': roc_auc})

learner = BertLearner.from_pretrained_model(databunch, 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz', metrics, device, logger=logger, 
                                            finetuned_wgts_path=None, 
                                            is_fp16=args['fp16'], loss_scale=args['loss_scale'], 
                                            multi_gpu=multi_gpu,  multi_label=True)

In [0]:
learner.fit(1, lr=args['learning_rate'], 
            schedule_type="warmup_cosine_hard_restarts")